### Extract timing files

In [7]:
#r "nuget: System.Data.SQLite, *-*"
#r "nuget: Microsoft.DotNet.Interactive.ExtensionLab, *-*"
#r "nuget: Dapper, *-*"

using System.IO;
using System.Text.Json;
using System.Text.Encodings.Web;
using System.Text.Unicode;
using System.Data.SQLite;
using Dapper;

var jsonSerializerOptions = new JsonSerializerOptions
{
    Encoder = JavaScriptEncoder.Create(UnicodeRanges.All),
    //WriteIndented = true
};

Installed Packages Dapper, 2.1.44 Microsoft.DotNet.Interactive.ExtensionLab, 1.0.0-beta.24229.4 System.Data.SQLite, 1.0.118

In [14]:
var dbPath = @"D:\DatabaseBackups\QuranDB\recitaion-timings\";

record RecitationTiming(long sura, long ayah, long time, string words);

bool SaveAsJson(string dbFilename){
    var dbFileFullName = dbPath + dbFilename + ".db";
    
    if (!File.Exists(dbFileFullName)) 
        return false;

    using (var connenction = new SQLiteConnection($"Data Source={dbFileFullName};Mode=Memory;Cache=Shared"))
    {
        var result = connenction.Query<RecitationTiming>("SELECT sura, ayah, time, words FROM timings ORDER BY sura, ayah");

        var arrayList = new List<object>();

        foreach (var item in result){
            var words = item.words.Split(',');
            var wordTimes = words.Select(w=>w.Split(':').Select(s=>Convert.ToInt32(s)));
            arrayList.Add(new object[]{ item.sura, item.ayah, item.time, wordTimes, "newLine" });
        }

        var resultJson = JsonSerializer.Serialize(arrayList, jsonSerializerOptions)
                            .Replace(",\"newLine\"],", "]," + Environment.NewLine)
                            .Replace(",\"newLine\"","");
    
        File.WriteAllText(dbPath + "extractedJson\\" + dbFilename + ".json", resultJson);
    }

    return true;
}

SaveAsJson("mishari_alafasy");

### Downlaod from quran.gov.bd

In [ ]:
using System.IO;
using System.Text.Json;
using System.Net;

var srcPath = @"D:\Faruque\Projects\quranpwa\src\assets\";
var quranDataJson = File.ReadAllText(srcPath + "quran-data.json");

public class QuranData
{
    public List<List<object>> suras { get; set; }
    public List<List<int>> hizb_quarters { get; set; }
    public List<List<int>> manzils { get; set; }
    public List<List<int>> rukus { get; set; }
    public List<List<int>> pages { get; set; }
    public List<List<object>> sajdas { get; set; }
    public List<List<int>> juzs { get; set; }
}

var quranData = JsonSerializer.Deserialize<QuranData>(quranDataJson);

record DownloadableFile(string url, string filename);

var downlaodableFiles = new List<DownloadableFile>();

for(int i = 0; i < quranData.suras.Count; i++){
    var sura = quranData.suras[i];
    var serial = i + 1;
    //var start = (int)sura[0];
    var ayas = int.Parse(sura[1].ToString());

    for(int j = 1; j <= ayas; j++){
        downlaodableFiles.Add( new($"{serial}/{serial}-{j}.mp3", $"{serial.ToString().PadLeft(3, '0')}{j.ToString().PadLeft(3, '0')}.mp3"));
    }
}

var urlBase = "http://quran.gov.bd/quran/Sound/bangla";
var downloadPath = @"D:\Faruque\Projects\QuranAudio\Mahbub_Subhan_quran.gov.bd";

foreach (var downlaodableFile in downlaodableFiles){
    string url = $"{urlBase}/{downlaodableFile.url}";
    string filePath = $"{downloadPath}\\{downlaodableFile.filename}";

    if (File.Exists(filePath))
        continue;

    using (var client = new WebClient())
    {
        client.DownloadFile(url, filePath);
    }
    Console.WriteLine($"Downloaded file: {downlaodableFile}");
}

Console.WriteLine($"Download completed");
